In [ ]:
!pip install selenium webdriver-manager

In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("user-data-dir=C:\\user-data\\kimfl")
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

driver = webdriver.Chrome(options=options)
driver.get("https://gcloud.csi.go.kr/cmq/main.do")



In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import UnexpectedAlertPresentException


# 이제 안전하게 쿠키를 가져옵니다.
selenium_cookies = driver.get_cookies()
session = requests.Session()
for cookie in selenium_cookies:
    session.cookies.set(cookie['name'], cookie['value'])

session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvList.do"
})

detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
target_rqst_nos = ["RQ-2025-242229"] # 테스트 대상
all_details = []

for rqst_no in target_rqst_nos:
    print(f"[{rqst_no}] 데이터 강제 정밀 추출 중...", end='\r')
    payload = {'rqltNo': rqst_no, 'pageMode': 'recv', 'searchKey': '1', 'searchDateKey': '1'}
    
    try:
        response = session.post(detail_url, data=payload, timeout=100)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            row_data = {"의뢰번호": rqst_no}
            
            # 페이지 내의 모든 th 태그를 대상으로 공백 제거 후 매칭 (가장 확실한 방법)
            all_th = soup.find_all("th")
            targets = ["접수번호", "최종진행상태", "의뢰기관", "공사명"]
            
            for t in targets:
                row_data[t] = "" # 초기화
                for th in all_th:
                    clean_th = th.get_text().replace(" ", "").replace("\n", "").replace("\t", "")
                    if t in clean_th:
                        # th 바로 다음에 오는 데이터 td를 찾음
                        target_td = th.find_next("td")
                        if target_td:
                            row_data[t] = target_td.get_text(strip=True)
                        break

            # --- 시험·검사종목 (ID testItemInfo 내부의 b 태그들 추출) ---
            row_data["시험·검사종목"] = ""
            test_items = soup.select("#testItemInfo b")
            if test_items:
                row_data["시험·검사종목"] = ", ".join([i.get_text(strip=True) for i in test_items])

            # --- 특정처리자 (처리기관이 한국건설품질시험원 주식회사인 경우) ---
            row_data["특정처리자"] = ""
            hist_div = soup.find(id="rqst_hist_div")
            if hist_div:
                rows = hist_div.select("tbody tr")
                for r in rows:
                    tds = r.find_all(["td", "th"])
                    if len(tds) >= 3:
                        agency = tds[1].get_text(strip=True)
                        if "한국건설품질시험원" in agency:
                            row_data["특정처리자"] = tds[2].get_text(strip=True)
                            break # 조건 만족 시 루프 종료

            all_details.append(row_data)
            print(f"[{rqst_no}] 추출 성공!                      ")
            
    except Exception as e:
        print(f"\n[{rqst_no}] 오류 발생: {e}")

    time.sleep(1)

# 결과 확인
df = pd.DataFrame(all_details)
display(df)
# df.to_excel("final_refined_data.xlsx", index=False)



[RQ-2025-242229] 추출 성공!                      


,의뢰번호,접수번호,최종진행상태,의뢰기관,공사명,시험·검사종목,특정처리자
0,RQ-2025-242229,,,,,,이동규
